# imports

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

### 导入数据

In [26]:
file_path1 = '附件1 近5年402家供应商的相关数据.xlsx'
df = pd.read_excel(file_path1)
df.head()

df_supply = pd.DataFrame({
    '供应商名称': df['供应商ID'],
    'mean':df.iloc[:,2:].mean(axis=1),
    'max':df.iloc[:,2:].max(axis=1),
    '材料分类':df['材料分类']
})
df_supply['转换值']=df_supply['材料分类'].map({'A':1/0.6,'B':1/0.66,'C':1/0.72})*df_supply['mean']
df_supply.describe()


,mean,max,转换值
count,402.000000,402.000000,402.000000
mean,60.421051,808.589552,92.349281
std,213.883556,3322.701535,329.708791
min,0.095833,1.000000,0.133102
25%,0.658333,20.000000,1.002736
50%,1.745833,100.000000,2.673611
75%,6.611458,400.000000,10.017361
max,2004.595833,36970.000000,3037.266414


## 变量

In [10]:
data = np.random.randint(2, size=402)


array([0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,

## 遗传算法类定义

In [ ]:
class GeneticAlgorithm:
    def __init__(self, population_size, generations, mutation_rate, data):
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.data = data  # 输入的数据
        self.population = []
        self.best_solution = None
        self.best_objective = float('inf')

    def initialize_population(self):
        # 随机初始化种群
        self.population = [self.random_solution() for _ in range(self.population_size)]

    def random_solution(self):
        # 根据输入数据随机生成一个解决方案
        return np.random.rand(len(self.data))

    def evaluate_fitness(self):
        for individual in self.population:
            objective = self.calculate_objective(individual)
            if self.check_constraints(individual):
                if objective < self.best_objective:
                    self.best_objective = objective
                    self.best_solution = individual
            else:
                # 如果不满足约束，可以给适应度一个很高的值
                objective += 1e6  # 或者其他惩罚值

    def calculate_objective(self, individual):
        # 计算目标函数
        # 这里可以根据具体问题实现目标函数
        objective_value = np.sum(individual)  # 示例：简单的目标函数
        return objective_value

    def check_constraints(self, individual):
        # 检查约束条件
        # 这里可以根据具体问题实现约束条件
        # 示例：假设个体的和必须小于 1
        return np.sum(individual) <= 1

    def calculate_cost(self, individual):
        # 根据目标函数计算成本
        return self.calculate_objective(individual)

    def select_parents(self):
        # 选择父代（轮盘赌选择）
        fitness_scores = [1 / self.calculate_cost(ind) for ind in self.population]
        total_fitness = sum(fitness_scores)
        probabilities = [f / total_fitness for f in fitness_scores]
        parents = np.random.choice(self.population, size=self.population_size, p=probabilities)
        return parents

    def crossover(self, parents):
        # 交叉生成子代
        children = []
        for i in range(0, self.population_size, 2):
            parent1, parent2 = parents[i], parents[i + 1]
            crossover_point = np.random.randint(1, len(parent1) - 1)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            children.append(child1)
            children.append(child2)
        return children

    def mutate(self, children):
        # 变异
        for child in children:
            if np.random.rand() < self.mutation_rate:
                mutation_point = np.random.randint(len(child))
                child[mutation_point] = np.random.rand()  # 随机变异
        return children

    def eliminate_weak_individuals(self):
        # 淘汰表现最差的个体
        fitness_scores = [self.calculate_cost(ind) for ind in self.population]
        sorted_population = sorted(zip(self.population, fitness_scores), key=lambda x: x[1])
        
        num_to_eliminate = len(self.population) // 2  # 淘汰一半个体
        retained_individuals = [individual for individual, _ in sorted_population[num_to_eliminate:]]
        
       
        self.population = retained_individuals  # 更新种群

    def optimize(self):
        self.initialize_population()
        for generation in range(self.generations):
            self.evaluate_fitness()
            print(f"Generation {generation}: Best objective = {self.best_objective:.4f}")

            parents = self.select_parents()
            children = self.crossover(parents)
            children = self.mutate(children)

            # 更新种群并淘汰表现最差的个体
            self.population = np.concatenate((self.population, children))  # 合并原种群与新子代
            self.eliminate_weak_individuals()  # 淘汰

        print(f"\nOptimization completed:")
        print(f"Best objective value: {self.best_objective:.4f}")
        return self.best_solution


## 使用示例

In [ ]:

# 使用示例
data = [...]  # 输入您的数据
ga = GeneticAlgorithm(population_size=100, generations=50, mutation_rate=0.01, data=data)
best_solution = ga.optimize()
print("Best solution:", best_solution)
